In [1]:
from elasticsearch import Elasticsearch

/Users/andersbarfod/anaconda/envs/plandata/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.23) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [4]:
import sys
sys.path.append('/Users/andersbarfod/Documents/python/')
import connections as con

In [12]:
df = con.sql_to_dataframe('DISTRIBUTION', 'select device_id, pressure, temperature ,battery, left(timestamp::text, 19) ts from "DecentFrames"')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17287 entries, 0 to 17286
Data columns (total 5 columns):
device_id      17287 non-null int64
pressure       17287 non-null float64
temperature    17287 non-null float64
battery        17287 non-null float64
ts             17287 non-null object
dtypes: float64(3), int64(1), object(1)
memory usage: 675.4+ KB


In [14]:
df.head()

,device_id,pressure,temperature,battery,ts
0,878,0.272217,17.3438,2.956,2018-08-29 10:14:54
1,878,0.272156,17.3406,2.956,2018-08-29 10:16:54
2,878,0.272095,17.3500,2.956,2018-08-29 10:17:56
3,878,0.272186,17.3625,2.956,2018-08-29 10:18:58
4,878,0.272156,17.3125,2.954,2018-08-29 10:19:56


# ES

In [15]:
es = Elasticsearch([{'host': '178.128.199.231', 'port': 9200}])

In [17]:
mapping = {
  "mappings": {
    "vandstand": { 
      "properties": { 
        "device_id": { "type": "integer"  }, 
        "pressure": { "type": "float"  },
        "temperature": { "type": "float"  }, 
        "battery": { "type": "float"  },
        "timestamp": { 
            "type": "date",
            "format": "yyyy-MM-dd HH:mm:ss"
        }
      }
    }
  }
}

In [18]:
# ignore 400 cause by IndexAlreadyExistsException when creating an index
es.indices.create(index='klima', ignore=400, body=mapping)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'klima'}

In [21]:
def df_to_es(row):
    
    data = {
        "device_id": row['device_id'],
        "pressure": row['pressure'],
        "temperature": row['temperature'], 
        "battery": row['battery'],
        "timestamp": row['ts']
    }
    
    res = es.index(index='klima', doc_type='vandstand', body=data)

In [22]:
df.apply(df_to_es, axis=1)

0        None
1        None
2        None
3        None
4        None
5        None
6        None
7        None
8        None
9        None
10       None
11       None
12       None
13       None
14       None
15       None
16       None
17       None
18       None
19       None
20       None
21       None
22       None
23       None
24       None
25       None
26       None
27       None
28       None
29       None
         ... 
17257    None
17258    None
17259    None
17260    None
17261    None
17262    None
17263    None
17264    None
17265    None
17266    None
17267    None
17268    None
17269    None
17270    None
17271    None
17272    None
17273    None
17274    None
17275    None
17276    None
17277    None
17278    None
17279    None
17280    None
17281    None
17282    None
17283    None
17284    None
17285    None
17286    None
Length: 17287, dtype: object

In [16]:
es.indices.delete(index='klima', ignore=[400, 404])

{'acknowledged': True}